In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from tensorflow.keras.layers import Dense, Activation, Softmax
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras import initializers
import numpy as np
import tensorflow as tf

In [10]:
from backprop_helper import sigmoid, get_model, sigmoid_jac, softmax, softmax_jac, MSE, MSE_grad

# Forward de la red
Dados los pesos y la estructura de una red neuronal con una softmax a la salida y un MSE como Loss calcular todo lo que se pide a continuación (No es comun usar MSE con la softmax pero a fines didácticos simplifica. Queda como ejercicio adicional resolver el mismo ejercicio pero con una categorical crossentropy a la saluda)

In [64]:
weights = np.load('weights_softmax_3_layers.npy', allow_pickle=True)
capas = ['Capa Densa 1 ws: 2x3', 'Capa Densa 1 biases:', 'Capa Densa 2 - ws: 3x3', 'Capa Densa 2 - biases', 'Capa Densa 3 - ws: 3x3', 'Capa Densa 3 - biases']
for i, layer in enumerate(weights):
    print(capas[i])
    print(layer)
    print()

Capa Densa 1 ws: 2x3
[[0.10820953 0.3432914  0.1744045 ]
 [0.05457611 0.54989725 0.34384015]]

Capa Densa 1 biases:
[-0.67943245 -0.00294854  0.15257952]

Capa Densa 2 - ws: 3x3
[[-0.7706185  -0.17550795]
 [-0.10197585  0.45046437]
 [ 0.00585397  0.3024927 ]]

Capa Densa 2 - biases
[-0.10661452 -0.34508756]

Capa Densa 3 - ws: 3x3
[[-0.49749678 -0.40208894 -0.85052264]
 [ 1.0619878   0.07141189  0.17314   ]]

Capa Densa 3 - biases
[-0.29359275 -0.7259881   0.578059  ]



La dimensión de entrada es 2

In [72]:
# Vector de entrada de ejemplo
X = np.array([[3.4, 2.1]])

D1_out = X.dot(weights[0]) + weights[1]


[[0.37510012 0.14510518 0.4797947 ]]


In [ ]:
A1_out = sigmoid(D1_out)
D2_out = A1_out.dot(weights[2]) + weights[3]
A2_out = sigmoid(D2_out)

D3_out = A2_out.dot(weights[4]) + weights[5]
P_est = softmax(D3_out)
print(P_est)

# MSE

### Implementar la función MSE en numpy:

In [69]:

print(X)
# Vector de salida de ejemplo
P_true = np.array([[1, 0, 0]])
print(P_true)

[[3.4 2.1]]
[[1 0 0]]


In [71]:
MSE(X, P_true)

ValueError: operands could not be broadcast together with shapes (1,2) (1,3) 

# Softmax 


### Implementar la función softmax en numpy:

In [38]:
# Vector de ejemplo
X = np.array([[3.4, 2.1]])

softmax(X)

array([[0.78583498, 0.21416502]])

### Implementar una función que devuelva el jacobiano de la softmax evaluado en un vector columna

In [39]:
# Vector de ejemplo
X = np.array([[3.4, 2.1]])

In [40]:
softmax_jac(X)

array([[ 0.16829836, -0.16829836],
       [-0.16829836,  0.16829836]])

In [12]:
sm = softmax(X)
np.diag(sm.reshape(-1)) - sm.reshape(-1, 1).dot(sm.reshape(1, -1))

array([[ 0.16829836, -0.16829836],
       [-0.16829836,  0.16829836]])

In [13]:
sm.reshape(-1, 1).dot(sm.reshape(1, -1))

array([[0.61753662, 0.16829836],
       [0.16829836, 0.04586665]])

In [15]:
softmax_jac(X)

array([[ 0.16829836, -0.16829836],
       [-0.16829836,  0.16829836]])

In [52]:
# model = get_model(lr = 1)

In [17]:
model = load_model('simple_model.hdf5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
D1 (Dense)                   (None, 3)                 9         
_________________________________________________________________
A1 (Activation)              (None, 3)                 0         
_________________________________________________________________
D2 (Dense)                   (None, 2)                 8         
_________________________________________________________________
A2 (Activation)              (None, 2)                 0         
_________________________________________________________________
D3 (Dense)                   (None, 3)                 9         
_________________________________________________________________
P_est (Activation)           (None, 3)                 0         
Total params: 26
Trainable params: 26
Non-trainable params: 0
____________________________________________________________

In [18]:
X = np.array([[3.4, 2.1]])

In [19]:
model.predict(X)

array([[0.37510014, 0.14510517, 0.4797947 ]], dtype=float32)

In [20]:
weights = model.get_weights()

In [31]:
D1_out = X.dot(weights[0]) + weights[1]
A1_out = sigmoid(D1_out)
D2_out = A1_out.dot(weights[2]) + weights[3]
A2_out = sigmoid(D2_out)

D3_out = A2_out.dot(weights[4]) + weights[5]
P_est = softmax(D3_out)
print(P_est)

[[0.37510012 0.14510518 0.4797947 ]]


In [22]:
# si el y es igual a la salida entonces los pesos no se modifican
P_true = np.array([[1, 0, 0]])

In [23]:
loss = MSE(P_true, P_est)
print(P_est)
print(loss)

[[0.37510012 0.14510518 0.4797947 ]]
0.2139194440951749


In [24]:
model.evaluate(X, P_true)

1/1 [==============================] - 0s 846us/step - loss: 0.2139


0.21391944587230682

In [32]:
backprop_D3_out = softmax_jac(D3_out).dot(MSE_grad(P_true, P_est).T)
A2_out.T.dot(backprop_D3_out.T)

array([[-0.05900569,  0.00456147,  0.05444422],
       [-0.08949681,  0.0069186 ,  0.08257822]])

In [33]:
backprop_D1_out = sigmoid_jac(D1_out).dot(weights[2].dot(sigmoid_jac(D2_out).dot(weights[4].dot(softmax_jac(D3_out).dot(MSE_grad(P_true, P_est).T)))))
X.T.dot(backprop_D1_out.T)

array([[ 0.01295024, -0.00409727, -0.00559341],
       [ 0.00799867, -0.00253067, -0.00345476]])

In [34]:
inputs = tf.constant(X)

with tf.GradientTape() as tape:
    preds = model(inputs)
    loss = model.loss(tf.constant(P_true), preds)

grads = tape.gradient(loss, model.get_layer("D1").trainable_variables)
print(grads[0])
# jacob = tape.jacobian(loss, model.get_layer("D3").trainable_variables)
# print(jacob)

tf.Tensor(
[[ 0.01295024 -0.00409727 -0.00559341]
 [ 0.00799868 -0.00253067 -0.00345476]], shape=(2, 3), dtype=float32)


# Para D1
$\theta_t = \theta_{t-1} - \mathrm{learning\_rate} * g_t$

In [37]:
(sigmoid_dif(D1_out).T*weights[2].dot(sigmoid_dif(D2_out).T*weights[4]*(-2*(y-y_)))).dot(X)

NameError: name 'y' is not defined

In [25]:
X*(sigmoid_dif(D1_out).T*weights[2].dot(sigmoid_dif(D2_out).T*weights[4]*(-2*(y-y_))))

NameError: name 'y' is not defined

### Derivada del MSE evaluada en y_

In [215]:
# Derivada de la loss respecto a y_
mse_der = -2*(y-y_)
print(mse_der)
prop_grad_y_ = mse_der

[[-3.48470968]]


### Delta para D3

In [216]:
# Gradiente de y_ respecto a cada parámetro
# Recordar que cuando derivo una capa dense respecto a cada parámetro, me da la entrada a esa cada. El Bias es como si entrarada con 1

y__grad_ws_d3 = A2_out
y__grad_bias_d3 = 1

delta_ws_d3 = y__grad_ws_d3*prop_grad_y_*lr
delta_bias_d3 = y__grad_bias_d3*prop_grad_y_*lr
print(delta_ws_d3, delta_bias_d3)

[[-2.26015587 -0.94712252 -1.49670461]] [[-3.48470968]]


In [217]:
# Pesos utlima capa modificados
new_weights[4].T - (weights[4].T - delta_ws_d3)

array([[7.67218125e-08, 5.60790036e-08, 1.47877194e-07]])

In [218]:
# Bias ultima capa modificado
new_weights[5].T - (weights[5].T - delta_bias_d3)

array([[9.2371895e-08]])

### A2

In [220]:
# Con que entro a A2? D2_out
# Necesito y__grad_in_d3 (Gradiente de y_ respecto a la entrada al bloque d3)
y__grad_in_d3 = weights[4]
prop_grad_d3 = y__grad_in_d3*prop_grad_y_
print(prop_grad_d3) # Entrada de D3
prop_grad_A2 = sigmoid_dif(D2_out).T*prop_grad_d3*lr
print(prop_grad_A2) # Entrada de A2

[[0.84538307]
 [1.28369178]
 [1.53521471]]
[[0.19267999]
 [0.25407076]
 [0.37617463]]


### D2

In [221]:
delta_ws_d2 = (prop_grad_A2*A1_out).T
delta_bias_d2 = prop_grad_A2.T

In [222]:
print(new_weights[2] - (weights[2] - delta_ws_d2))
print(new_weights[3] - (weights[3] - delta_bias_d2))

[[-6.25856622e-09 -5.46572199e-09  1.02110236e-08]
 [-1.62406742e-08 -1.44461989e-08 -1.64342799e-08]]
[[-1.48808340e-08  1.29551381e-09  1.10791643e-09]]


## A1

In [223]:
d2_out_grad_in_d2 = weights[2]
prop_grad_d2 = d2_out_grad_in_d2.dot(prop_grad_A2)
print(prop_grad_d2) 
prop_grad_A1 = sigmoid_dif(D1_out).T*prop_grad_d2*lr
print(prop_grad_A1) 

[[-0.19309713]
 [ 0.082746  ]]
[[-0.04419043]
 [ 0.02066716]]


### D1

In [224]:
prop_grad_A1

array([[-0.04419043],
       [ 0.02066716]])

In [249]:
delta_ws_d1 = (X*prop_grad_A1).T
delta_bias_d1 = prop_grad_A1.T
print(delta_ws_d1)

[[-0.15024746  0.07026835]
 [-0.0927999   0.04340104]]


In [250]:
print(new_weights[0] - (weights[0] - delta_ws_d1))
print(new_weights[1] - (weights[1] - delta_bias_d1))

[[ 1.94892052e-08 -8.69921603e-09]
 [-1.77648721e-08 -9.97487437e-09]]
[[-2.40702032e-08 -4.74994019e-09]]


In [393]:
A3_model = Model(model.input, model.get_layer('A3').input)

In [405]:
A3_model(inputs)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 0.19870196,  0.01208979, -0.80531573]], dtype=float32)>

In [22]:
model.get_layer("D3").trainable_variables

[<tf.Variable 'D3_1/kernel:0' shape=(2, 3) dtype=float32, numpy=
 array([[ 0.3395361 , -0.58271265,  0.21878819],
        [-0.0810146 , -0.3692061 , -0.07064136]], dtype=float32)>,
 <tf.Variable 'D3_1/bias:0' shape=(3,) dtype=float32, numpy=array([ 0.66752887, -0.21773213, -0.14207424], dtype=float32)>]

In [21]:
model = load_model('simple_model.hdf5')
inputs = tf.constant(X)

with tf.GradientTape() as tape:
    preds = model(inputs)
    loss = model.loss(tf.constant(P_true), preds)

grads = tape.gradient(loss, model.get_layer("D1").trainable_variables)
print(grads)
# grads = tape.gradient(loss, model.get_layer("D3").trainable_variables)
# print(grads)

[<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-0.01626888, -0.00131034,  0.00017072],
       [-0.01004843, -0.00080932,  0.00010545]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-4.7849659e-03, -3.8539272e-04,  5.0213028e-05], dtype=float32)>]


In [22]:
X.T.dot(sigmoid_jac(D1_out).dot(weights[2].dot(sigmoid_jac(D2_out).dot(weights[4].dot(softmax_jac(D3_out).dot((-2*(P_est-P_true)).T))))).T)

array([[ 1.24122276e-02,  1.46758699e-03, -9.68974365e-05],
       [ 7.66637585e-03,  9.06450788e-04, -5.98484167e-05]])

In [97]:
X.T.dot(sigmoid_jac(D1_out).dot(weights[2].dot(sigmoid_jac(D2_out).dot(weights[4].dot(softmax_jac(D3_out).dot((-2*(P_est-P_true)).T))))).T)

array([[-0.00063273,  0.00029997,  0.00871013],
       [-0.00039081,  0.00018528,  0.00537978]])

In [362]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
D1 (Dense)                   (None, 3)                 9         
_________________________________________________________________
A1 (Activation)              (None, 3)                 0         
_________________________________________________________________
D2 (Dense)                   (None, 2)                 8         
_________________________________________________________________
A2 (Activation)              (None, 2)                 0         
_________________________________________________________________
D3 (Dense)                   (None, 3)                 9         
_________________________________________________________________
A3 (Activation)              (None, 3)                 0         
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________

In [178]:
model.trainable_weights

[<tf.Variable 'dense_63_4/kernel:0' shape=(2, 2) dtype=float32, numpy=
 array([[-0.0132913 ,  0.39277452],
        [ 0.61490476, -0.11834475]], dtype=float32)>,
 <tf.Variable 'dense_63_4/bias:0' shape=(2,) dtype=float32, numpy=array([ 0.9016673 , -0.33314416], dtype=float32)>,
 <tf.Variable 'dense_64_4/kernel:0' shape=(2, 3) dtype=float32, numpy=
 array([[-0.4778273 ,  0.3336253 , -0.03871878],
        [ 0.07612087,  0.27854922, -0.02718207]], dtype=float32)>,
 <tf.Variable 'dense_64_4/bias:0' shape=(3,) dtype=float32, numpy=array([ 0.36176467, -0.86578524, -0.10165475], dtype=float32)>,
 <tf.Variable 'dense_65_4/kernel:0' shape=(3, 1) dtype=float32, numpy=
 array([[0.47252566],
        [0.73365825],
        [0.7718087 ]], dtype=float32)>,
 <tf.Variable 'dense_65_4/bias:0' shape=(1,) dtype=float32, numpy=array([1.4085784], dtype=float32)>]

In [179]:
model.output

<tf.Tensor 'dense_65_4/Identity:0' shape=(None, 1) dtype=float32>

In [20]:
from tensorflow.keras import backend as K

In [21]:
K.gradients(model.output, model.trainable_weights)

RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

In [ ]:
tf.GradientTape()